In [1]:
from neo4j import GraphDatabase

# 接続、初期化

In [2]:
uri = 'bolt://neo4j:7687'
user = 'neo4j'
password = 'neo4j'
driver = GraphDatabase.driver(uri, auth=(user, password))

In [3]:
def clear_db(tx):
    tx.run('MATCH (n) DETACH DELETE n')

In [4]:
with driver.session() as session:
    session.write_transaction(clear_db)

# 追加

In [5]:
def add_friend(tx, name, friend_name=None):
    if not friend_name:
        tx.run('CREATE (p:Person {name: $name}) RETURN p', name=name)
    else:
        tx.run(
            'MATCH (p:Person {name: $name})'
            'CREATE (p)-[:FRIEND]->(:Person {name: $friend_name})',
            name=name, friend_name=friend_name)

In [6]:
def get_friend(tx, name):
    friends = []
    for record in tx.run('MATCH (p:Person {name: $name})-[:FRIEND]->(yourfriends)'
                         'RETURN p,yourfriends', name=name):
        friends.append(record)
    return friends

In [7]:
with driver.session() as session:
    session.write_transaction(add_friend, 'MasterU')
    for f in ['Mark', 'Kent']:
        session.write_transaction(add_friend, 'MasterU', f)

In [8]:
with driver.session() as session:
    friends = session.read_transaction(get_friend, 'MasterU')

In [9]:
friends

[<Record p=<Node id=0 labels=frozenset({'Person'}) properties={'name': 'MasterU'}> yourfriends=<Node id=1 labels=frozenset({'Person'}) properties={'name': 'Mark'}>>,
 <Record p=<Node id=0 labels=frozenset({'Person'}) properties={'name': 'MasterU'}> yourfriends=<Node id=2 labels=frozenset({'Person'}) properties={'name': 'Kent'}>>]

In [10]:
type(friends[0])

neo4j.data.Record

In [11]:
friends[0]['p']

<Node id=0 labels=frozenset({'Person'}) properties={'name': 'MasterU'}>

In [12]:
friends[0]['yourfriends']

<Node id=1 labels=frozenset({'Person'}) properties={'name': 'Mark'}>